In [1]:
import os
import pandas as pd
from itertools import accumulate

## 获取数据集位置

In [2]:
floder_path=r"D:\trunz\Documents\DataScience\R_visualization\Data-Visualization_Teamwork\Data Set\北京空气质量"

filesList=[]
filesNum=[]
for root, dirs, files in os.walk(floder_path):
    filesNum.append(len(files))
    for file in files:
        if file.endswith(".csv"):
            file_path=os.path.join(root, file)
            filesList.append(file_path)

### 得到数据集路径与不同年份的index

In [3]:
filesNum=filesNum[1:-1]
filesNum

[627, 728, 732, 730, 730, 728, 730, 599, 511, 730]

In [4]:
fileindex=[0]
for num in filesNum:
    fileindex.append(fileindex[-1]+num)
fileindex

[0, 627, 1355, 2087, 2817, 3547, 4275, 5005, 5604, 6115, 6845]

## Load数据集并整合不同表格，得到逐年份的分时数据

In [5]:
DataList=[]
for i in range(len(filesNum)):
    df_all=pd.DataFrame()
    for file in filesList[fileindex[i]:fileindex[i+1]]:
        if os.path.getsize(file)==0:
            continue
        df_temp=pd.read_csv(file)
        try:
            df_temp=df_temp[df_temp['type'].isin(['PM2.5','PM10','AQI','SO2','NO2','O3','CO'])]
        except Exception as e:
            continue
        df_all=pd.concat([df_all,df_temp])
        df_all=df_all.sort_values(by=['date','hour'])
    DataList.append(df_all)

In [6]:
for data in DataList:
    print(data.columns)

Index(['date', 'hour', 'type', '东四', '天坛', '官园', '万寿西宫', '奥体中心', '农展馆', '万柳',
       '北部新区', '植物园', '丰台花园', '云岗', '古城', '房山', '大兴', '亦庄', '通州', '顺义', '昌平',
       '门头沟', '平谷', '怀柔', '密云', '延庆', '定陵', '八达岭', '密云水库', '东高村', '永乐店', '榆垡',
       '琉璃河', '前门', '永定门内', '西直门北', '南三环', '东四环'],
      dtype='object')
Index(['date', 'hour', 'type', '东四', '天坛', '官园', '万寿西宫', '奥体中心', '农展馆', '万柳',
       '北部新区', '植物园', '丰台花园', '云岗', '古城', '房山', '大兴', '亦庄', '通州', '顺义', '昌平',
       '门头沟', '平谷', '怀柔', '密云', '延庆', '定陵', '八达岭', '密云水库', '东高村', '永乐店', '榆垡',
       '琉璃河', '前门', '永定门内', '西直门北', '南三环', '东四环'],
      dtype='object')
Index(['date', 'hour', 'type', '东四', '天坛', '官园', '万寿西宫', '奥体中心', '农展馆', '万柳',
       '北部新区', '植物园', '丰台花园', '云岗', '古城', '房山', '大兴', '亦庄', '通州', '顺义', '昌平',
       '门头沟', '平谷', '怀柔', '密云', '延庆', '定陵', '八达岭', '密云水库', '东高村', '永乐店', '榆垡',
       '琉璃河', '前门', '永定门内', '西直门北', '南三环', '东四环'],
      dtype='object')
Index(['date', 'hour', 'type', '东四', '天坛', '官园', '万寿西宫', '奥体中心', '农展馆', '万柳'

In [7]:
output_dir='output_data'
os.makedirs(output_dir,exist_ok=True)

# 保存 DataList 中的每个 DataFrame 为单独的 CSV 文件
for i, df in enumerate(DataList):
    output_file=os.path.join(output_dir,f'Beijing_AirQuality_{i+2014}_hours.csv')
    df.to_csv(output_file,index=False)
    print(f"已保存文件: {output_file}")

已保存文件: output_data\Beijing_AirQuality_2014_hours.csv
已保存文件: output_data\Beijing_AirQuality_2015_hours.csv
已保存文件: output_data\Beijing_AirQuality_2016_hours.csv
已保存文件: output_data\Beijing_AirQuality_2017_hours.csv
已保存文件: output_data\Beijing_AirQuality_2018_hours.csv
已保存文件: output_data\Beijing_AirQuality_2019_hours.csv
已保存文件: output_data\Beijing_AirQuality_2020_hours.csv
已保存文件: output_data\Beijing_AirQuality_2021_hours.csv
已保存文件: output_data\Beijing_AirQuality_2022_hours.csv
已保存文件: output_data\Beijing_AirQuality_2023_hours.csv


## 异常值缺失值处理 & 获得不同污染物的逐日和逐月逐年数据

In [8]:
for i,data in enumerate(DataList):
    for gas in ['AQI','CO','NO2','O3','PM10','PM2.5','SO2']:
        df=data[data['type']==gas]
    
        # 处理异常值和缺失值
        for station in df.columns[3:]:
            anomalous_index=df[df[station]>=1200].index
            df=df.drop(anomalous_index)
    
            # 检测是否存在缺失值
            missing_count=df[station].isnull().sum()
            if missing_count>0:
                # 使用插值法填充缺失值 
                df[station] = df[station].interpolate(method='linear')
                # 对仍有缺失值的部分，使用前向填充和后向填充补齐
                df[station] = df[station].fillna(method='ffill').fillna(method='bfill')
            
    
        # 逐小时数据导出
        output_dir='output_data'
        os.makedirs(output_dir,exist_ok=True)
        
        # 逐日数据导出
        numeric_columns = df.columns[3:]
        df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')
        daily_avg = df.groupby('date')[numeric_columns].mean().reset_index()  
        
        output_file=os.path.join(output_dir,f'Beijing_AirQuality_{i+2014}{gas}_days.csv')
        daily_avg.to_csv(output_file,index=False,encoding='utf-8-sig')
        print(f"已保存文件: {output_file}")
        
        # 逐月数据导出
        df['month'] = df['date'].astype(str).str[:6]
        monthly_avg = df.groupby('month')[numeric_columns].mean().reset_index()
        
        output_file=os.path.join(output_dir,f'Beijing_AirQuality_{i+2014}{gas}_months.csv')
        monthly_avg.to_csv(output_file,index=False,encoding='utf-8-sig')
        print(f"已保存文件: {output_file}")
    

已保存文件: output_data\Beijing_AirQuality_2014AQI_days.csv
已保存文件: output_data\Beijing_AirQuality_2014AQI_months.csv


C:\Users\trunz\AppData\Local\Temp\ipykernel_13844\1634777770.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[station] = df[station].fillna(method='ffill').fillna(method='bfill')
C:\Users\trunz\AppData\Local\Temp\ipykernel_13844\1634777770.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[station] = df[station].fillna(method='ffill').fillna(method='bfill')
C:\Users\trunz\AppData\Local\Temp\ipykernel_13844\1634777770.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[station] = df[station].fillna(method='ffill').fillna(method='bfill')
C:\Users\trunz\AppData\Local\Temp\ipykernel_13844\1634777770.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill(

已保存文件: output_data\Beijing_AirQuality_2014CO_days.csv
已保存文件: output_data\Beijing_AirQuality_2014CO_months.csv
已保存文件: output_data\Beijing_AirQuality_2014NO2_days.csv
已保存文件: output_data\Beijing_AirQuality_2014NO2_months.csv


C:\Users\trunz\AppData\Local\Temp\ipykernel_13844\1634777770.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[station] = df[station].fillna(method='ffill').fillna(method='bfill')
C:\Users\trunz\AppData\Local\Temp\ipykernel_13844\1634777770.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[station] = df[station].fillna(method='ffill').fillna(method='bfill')
C:\Users\trunz\AppData\Local\Temp\ipykernel_13844\1634777770.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[station] = df[station].fillna(method='ffill').fillna(method='bfill')
C:\Users\trunz\AppData\Local\Temp\ipykernel_13844\1634777770.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill(

已保存文件: output_data\Beijing_AirQuality_2014O3_days.csv
已保存文件: output_data\Beijing_AirQuality_2014O3_months.csv


...

C:\Users\trunz\AppData\Local\Temp\ipykernel_13844\1634777770.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[station] = df[station].fillna(method='ffill').fillna(method='bfill')
C:\Users\trunz\AppData\Local\Temp\ipykernel_13844\1634777770.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[station] = df[station].fillna(method='ffill').fillna(method='bfill')
C:\Users\trunz\AppData\Local\Temp\ipykernel_13844\1634777770.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[station] = df[station].fillna(method='ffill').fillna(method='bfill')
C:\Users\trunz\AppData\Local\Temp\ipykernel_13844\1634777770.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill(

已保存文件: output_data\Beijing_AirQuality_2023NO2_days.csv
已保存文件: output_data\Beijing_AirQuality_2023NO2_months.csv
已保存文件: output_data\Beijing_AirQuality_2023O3_days.csv
已保存文件: output_data\Beijing_AirQuality_2023O3_months.csv
已保存文件: output_data\Beijing_AirQuality_2023PM10_days.csv
已保存文件: output_data\Beijing_AirQuality_2023PM10_months.csv


C:\Users\trunz\AppData\Local\Temp\ipykernel_13844\1634777770.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[station] = df[station].fillna(method='ffill').fillna(method='bfill')
C:\Users\trunz\AppData\Local\Temp\ipykernel_13844\1634777770.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[station] = df[station].fillna(method='ffill').fillna(method='bfill')
C:\Users\trunz\AppData\Local\Temp\ipykernel_13844\1634777770.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[station] = df[station].fillna(method='ffill').fillna(method='bfill')
C:\Users\trunz\AppData\Local\Temp\ipykernel_13844\1634777770.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill(

已保存文件: output_data\Beijing_AirQuality_2023PM2.5_days.csv
已保存文件: output_data\Beijing_AirQuality_2023PM2.5_months.csv
已保存文件: output_data\Beijing_AirQuality_2023SO2_days.csv
已保存文件: output_data\Beijing_AirQuality_2023SO2_months.csv


C:\Users\trunz\AppData\Local\Temp\ipykernel_13844\1634777770.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[station] = df[station].fillna(method='ffill').fillna(method='bfill')
C:\Users\trunz\AppData\Local\Temp\ipykernel_13844\1634777770.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[station] = df[station].fillna(method='ffill').fillna(method='bfill')


In [9]:
# 逐年数据
for gas in ['CO']:  #['AQI','CO','NO2','O3','PM10','PM2.5','SO2']:
    yearly_data=pd.DataFrame()
    monthly_data=pd.DataFrame()
    for year in range(2014,2024):
        file_path=os.path.join(output_dir,f'Beijing_AirQuality_{year}{gas}_months.csv')
        df=pd.read_csv(file_path)
        monthly_data=pd.concat([monthly_data,df],ignore_index=True,join='outer')
    
    monthly_data['year']=monthly_data['month'].astype(str).str[:4]
    yearly_avg=monthly_data.groupby('year')[numeric_columns].mean().reset_index()
    
yearly_avg

,year,东城东四,东城天坛,西城官园,西城万寿西宫,朝阳奥体中心,朝阳农展馆,海淀万柳,海淀四季青,丰台小屯,...,通州永顺,通州东关,大兴黄村,大兴旧宫,亦庄开发区,京东南区域点,门头沟双峪,门头沟三家店,房山良乡,房山燕山
0,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2021,0.714527,0.707298,0.695912,0.685098,0.705393,0.683939,0.652452,0.648014,0.677857,...,0.640479,0.655228,0.714681,0.609693,0.657653,0.726481,0.593865,0.680987,0.687710,0.554432
8,2022,0.532950,0.517629,0.525197,0.576287,0.504432,0.534359,0.508494,0.522915,0.504206,...,0.520060,0.533464,0.553837,0.537693,0.575905,0.573801,0.432758,0.462883,0.516061,0.431448
9,2023,0.563976,0.525983,0.520673,0.586694,0.517075,0.528590,0.535848,0.536877,0.533178,...,0.582046,0.540635,0.565995,0.519819,0.582404,0.568795,0.439525,0.402178,0.519238,0.420423
